In [1]:
import cv2
import random
import mediapipe as mp

mp_hands = mp.solutions.hands

mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

cap = cv2.VideoCapture(0)
hands = mp_hands.Hands()


frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

circle_x = random.randint(25,frame_width-25)
circle_y = frame_height // 2 - frame_height // 2 - 25
fallen = False
counter = 0

color_r = random.randint(0,255)
color_g = random.randint(0,255)
color_b = random.randint(0,255)

speed = 10

score = 0
increase = False

game_over = False

while cap.isOpened():
    state, frame = cap.read()
    
    if state:

        frame = cv2.flip(frame,1)
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        if circle_y >= frame_height+25 and not fallen:
            cv2.putText(frame,"GAME OVER!",(frame_width//2-100,frame_height//2),cv2.FONT_ITALIC,1,(color_r,color_g,color_b),3)
            game_over = True

        if fallen:
            circle_x = random.randint(25,frame_width-25)
            circle_y = frame_height // 2 - frame_height // 2 - 25
            color_r = random.randint(0,255)
            color_g = random.randint(0,255)
            color_b = random.randint(0,255)
            fallen = False
        else:
            circle_y+=speed

        cv2.circle(frame,(circle_x,circle_y),20,(color_r,color_g,color_b),5)

        results = hands.process(frame_rgb)

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(frame,hand_landmarks,mp_hands.HAND_CONNECTIONS)

                center_x = int(hand_landmarks.landmark[8].x*frame.shape[1])
                center_y = int(hand_landmarks.landmark[8].y*frame.shape[0])
                cv2.circle(frame,(center_x,center_y),20,(color_r,color_g,color_b),5)

            if center_x in range(circle_x-25,circle_x+25) and center_y in range(circle_y-25,circle_y+25):
                fallen = True
                increase = False
                score+=1

        if score%5==0 and not increase and score!=0:
            speed+=1
            increase = True

        cv2.putText(frame,f"Score: {score}",(30,50),cv2.FONT_ITALIC,1,(color_r,color_g,color_b),3)
        cv2.imshow("FallingCircles", frame)

        k = cv2.waitKey(1)
        if k == ord("q"):
            break
        elif k == ord("r") and game_over:
            circle_x = random.randint(25,frame_width-25)
            circle_y = frame_height // 2 - frame_height // 2 - 25
            fallen = False
            color_r = random.randint(0,255)
            color_g = random.randint(0,255)
            color_b = random.randint(0,255)
            speed = 10
            score = 0
            increase = False
            game_over = False
    else:
        break
cap.release()
cv2.destroyAllWindows()
